## A. Funciones.

In [ ]:
#i. PSI.
#3. Funciones.
#i. PSI.
def psi(expected, actual, buckets=10):

    def psi_formula(expected_prop, actual_prop):
        result = (actual_prop - expected_prop) * np.log(actual_prop / expected_prop)
        return result

    expected_not_null = expected.dropna()
    actual_not_null = actual.dropna()

    bin_edges = pd.qcut(expected_not_null, q=buckets, duplicates='drop').unique()
    bin_edges2 = [edge.left for edge in bin_edges] + [edge.right for edge in bin_edges]
    breakpoints = sorted(list(set(bin_edges2)))

    expected_counts, _ = np.histogram(expected_not_null, bins=breakpoints)
    actual_counts, _ = np.histogram(actual_not_null, bins=breakpoints)

    expected_prop = expected_counts / len(expected_not_null)
    actual_prop = actual_counts / len(actual_not_null)

    psi_not_null = psi_formula(expected_prop, actual_prop).sum()

    psi_null = 0

    if expected.isnull().sum() > 0 and actual.isnull().sum() > 0 :
      expected_null_percentage = expected.isnull().mean()
      actual_null_percentage = actual.isnull().mean()
      psi_null = psi_formula(expected_null_percentage, actual_null_percentage)

    return psi_not_null + psi_null

In [ ]:
#ii. Evaluación de Ganancia (LGBM).
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True


In [ ]:
#iii. Probabilidad de Ganancia (LGBM).
def ganancia_prob(y_pred, y_true, prop = 1):
  ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
  return ganancia[y_pred >= 0.025].sum() / prop

In [ ]:
#iv. Probabilidad de Ganancia (Random Forest).
def ganancia_prob_rf(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop

In [ ]:
#v. Diferentes funciones y métodos para corregir el efecto de la inflación.
def drift_uva(dataset, campos_monetarios, tb_indices):
    print("inicio drift_UVA()")
    dataset = dataset.merge(tb_indices[['foto_mes', 'UVA']], on='foto_mes', how='left')
    for campo in campos_monetarios:
        dataset[campo] *= dataset['UVA']
    dataset.drop(columns=['UVA'], inplace=True)
    print("fin drift_UVA()")
    return dataset

def drift_deflacion(dataset, campos_monetarios, tb_indices):
    print("inicio drift_deflacion()")
    dataset = dataset.merge(tb_indices[['foto_mes', 'IPC']], on='foto_mes', how='left')
    for campo in campos_monetarios:
        dataset[campo] *= dataset['IPC']
    dataset.drop(columns=['IPC'], inplace=True)
    print("fin drift_deflacion()")
    return dataset

# Función para estandarizar datos
def drift_estandarizar(dataset, campos_drift):
    print("inicio drift_estandarizar()")
    for campo in campos_drift:
        dataset[campo + "_normal"] = dataset.groupby('foto_mes')[campo].transform(lambda x: (x - x.mean()) / x.std())
        dataset.drop(columns=[campo], inplace=True)
    print("fin drift_estandarizar()")
    return dataset

In [ ]:
#vi. Adaptación de la función de ganancia para Random Forest.
def rf_gan_eval(y_true, y_pred_proba, weights):
    # Define los parámetros de ganancia y costo
    ganancia_acierto = 7800  # Ajusta según tus datos
    costo_estimulo = 200  # Ajusta según tus datos
    
    # Calcular la ganancia en base a las predicciones y los pesos
    ganancia = np.where(weights == 1.00002, ganancia_acierto, 0) - np.where(weights < 1.00002, costo_estimulo, 0)
    
    # Ordenar las ganancias de acuerdo a las probabilidades de la clase positiva
    ganancia = ganancia[np.argsort(y_pred_proba)[::-1]]
    
    # Cálculo de ganancia acumulada y obtención de la máxima ganancia
    ganancia_acumulada = np.cumsum(ganancia)
    max_ganancia = np.max(ganancia_acumulada)
    
    return max_ganancia

In [ ]:
#vi. Funciones para limpieza de variables rotas.
def corregir_interpolar(dataset, campo, meses):
    """
    Corrige valores interpolando entre datos adyacentes, respetando clientes.
    """
    # Crear columnas con valores anteriores (lag) y siguientes (lead) dentro de cada cliente
    dataset['v1'] = dataset.groupby('numero_de_cliente')[campo].shift(1)  # Valor anterior
    dataset['v2'] = dataset.groupby('numero_de_cliente')[campo].shift(-1)  # Valor siguiente

    # Calcular el promedio entre lag y lead
    dataset['promedio'] = dataset[['v1', 'v2']].mean(axis=1, skipna=True)

    # Sustituir los valores en los meses especificados con el promedio calculado
    dataset.loc[dataset['foto_mes'].isin(meses), campo] = dataset.loc[
        dataset['foto_mes'].isin(meses), 'promedio'
    ]

    # Eliminar columnas temporales
    dataset.drop(columns=['v1', 'v2', 'promedio'], inplace=True)

    return dataset

def corregir_mice(dataset, campo, meses):
    """Usa imputación de datos tipo MICE para corregir valores."""
    # Simple implementación para imputar valores con muestras similares
    imputador = SimpleImputer(strategy="most_frequent")
    dataset.loc[dataset['foto_mes'].isin(meses), campo] = imputador.fit_transform(
        dataset.loc[dataset['foto_mes'].isin(meses), [campo]]
    )
    return dataset

def asignar_na(dataset, campo, meses):
    """Asigna NA a los valores de un campo en los meses especificados."""
    if campo in dataset.columns:
        dataset.loc[dataset['foto_mes'].isin(meses), campo] = np.nan
    return dataset

def corregir_atributo(dataset, campo, meses, metodo):
    """Corrige un campo dado un método."""
    if campo not in dataset.columns:
        return dataset  # Si no existe el campo, no hace nada

    if metodo == "MachineLearning":
        dataset = asignar_na(dataset, campo, meses)
    elif metodo == "EstadisticaClasica":
        dataset = corregir_interpolar(dataset, campo, meses)
    elif metodo == "MICE":
        dataset = corregir_mice(dataset, campo, meses)
    return dataset

def corregir_rotas(dataset, metodo):
    """Aplica las correcciones a los campos dañados."""
    print("Inicio de corregir_rotas()")
    
    atributos_y_meses = [
        ("active_quarter", [202006]),  # 1
        ("internet", [202006]),  # 2
        ("mrentabilidad", [201905, 201910, 202006]),  # 3
        ("mrentabilidad_annual", [201905, 201910, 202006]),  # 4
        ("mcomisiones", [201905, 201910, 202006]),  # 5
        ("mactivos_margen", [201905, 201910, 202006]),  # 6
        ("mpasivos_margen", [201905, 201910, 202006]),  # 7
        ("mcuentas_saldo", [202006]),  # 8
        ("ctarjeta_debito_transacciones", [202006]),  # 9
        ("mautoservicio", [202006]),  # 10
        ("ctarjeta_visa_transacciones", [202006]),  # 11
        ("mtarjeta_visa_consumo", [202006]),  # 12
        ("ctarjeta_master_transacciones", [202006]),  # 13
        ("mtarjeta_master_consumo", [202006]),  # 14
        ("ctarjeta_visa_debitos_automaticos", [201904]),  # 15
        ("mttarjeta_visa_debitos_automaticos", [201904]),  # 16
        ("ccajeros_propios_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 17
        ("mcajeros_propios_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 18
        ("ctarjeta_visa_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 19
        ("mtarjeta_visa_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 20
        ("ctarjeta_master_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 21
        ("mtarjeta_master_descuentos", [201910, 202002, 202006, 202009, 202010, 202102]),  # 22
        ("ccomisiones_otras", [201905, 201910, 202006]),  # 23
        ("mcomisiones_otras", [201905, 201910, 202006]),  # 24
        ("cextraccion_autoservicio", [202006]),  # 25
        ("mextraccion_autoservicio", [202006]),  # 26
        ("ccheques_depositados", [202006]),  # 27
        ("mcheques_depositados", [202006]),  # 28
        ("ccheques_emitidos", [202006]),  # 29
        ("mcheques_emitidos", [202006]),  # 30
        ("ccheques_depositados_rechazados", [202006]),  # 31
        ("mcheques_depositados_rechazados", [202006]),  # 32
        ("ccheques_emitidos_rechazados", [202006]),  # 33
        ("mcheques_emitidos_rechazados", [202006]),  # 34
        ("tcallcenter", [202006]),  # 35
        ("ccallcenter_transacciones", [202006]),  # 36
        ("thomebanking", [202006]),  # 37
        ("chomebanking_transacciones", [201910, 202006]),  # 38
        ("ccajas_transacciones", [202006]),  # 39
        ("ccajas_consultas", [202006]),  # 40
        ("ccajas_depositos", [202006, 202105]),  # 41
        ("ccajas_extracciones", [202006]),  # 42
        ("ccajas_otras", [202006]),  # 43
        ("catm_trx", [202006]),  # 44
        ("matm", [202006]),  # 45
        ("catm_trx_other", [202006]),  # 46
        ("matm_other", [202006])  # 47
    ]

    for atributo, meses in atributos_y_meses:
        dataset = corregir_atributo(dataset, atributo, meses, metodo)

    print("Fin de corregir_rotas()")
    return dataset

def eliminar_atributo(dataset, atributo):
    """Elimina un atributo del dataset."""
    if atributo in dataset.columns:
        dataset.drop(columns=[atributo], inplace=True)
    return dataset